In [3]:
import numpy as np
import math
import random
from copy import deepcopy
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [53]:
class SimulatedAnnealingCV:
    
    def __init__(self, model, param_grid, scoring='accuracy', num_iterations=50, cv=5,early_stopping=False,k=None):
        self.model = model
        self.param_grid = param_grid
        self.X=None
        self.y=None
        self.scoring = scoring
        self.num_iterations = num_iterations
        self.cv = cv
        self.best_params=None
        self.best_score=float('-inf')
        self.k=k
        self.early_stopping=early_stopping
        
    def fit(self, X, y):
        self.X=X
        self.y=y
        
        params = self.generate_parameters()
        score = self.evaluate_params(params)
        
        no_improvement_count = 0
        if self.early_stopping and k == None:
            k=int(self.num_iterations*0.1)
            
        for i in range(1,self.num_iterations):  
            new_params = self.generate_parameters()
            new_score = self.evaluate_params(params)
            
            #print("new score",new_score,"best score",self.best_score)
            
            if new_score > score:
                score=new_score
                params=deepcopy(new_params)
                if new_score > self.best_score:
                    self.best_score=new_score
                    self.best_params = deepcopy(new_params)
                no_improvement_count = 0
            else:
                p = 1/i**0.5
                q=random.random()
                if q<p:
                    score=new_score
                    params=deepcopy(new_params)
                    no_improvement_count = 0
                else:
                    no_improvement_count+=1
                    
            if self.early_stopping and no_improvement_count >= k:
                break

    def generate_parameters(self):
        return {param : random.choice(value_list) for param,value_list in self.param_grid.items()}
        #TODO: different strategy for choosing neighbor of current params

    def evaluate_params(self,params):
        self.model.set_params(**params)
        scores = cross_val_score(self.model, self.X, self.y, cv=self.cv, scoring=self.scoring)
        return scores.mean()

In [54]:
param_grid={
    'max_depth' : [4,5,6],
    'criterion' : ['gini','entropy']
}

In [55]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [56]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()

In [57]:
sa=SimulatedAnnealingCV(model,param_grid=param_grid)

In [58]:
sa.fit(X,y)

In [59]:
sa.best_score

0.9666666666666668

In [48]:
#TODO: test on different data

In [49]:
#TODO: test on different param_grid

In [50]:
#TODO: show execution time 

In [52]:
#TODO: compare to local search ?

In [ ]:
#TODO: earlier stopping ? (added)